# Tagging using Mistral 7B via Ollama

In [1]:
from pydantic import BaseModel
from llama_index.program import LLMTextCompletionProgram
from llama_index.llms import Ollama
from llama_index.prompts import PromptTemplate
from llama_index.output_parsers import PydanticOutputParser

Define output schema

In [2]:
class Tag(BaseModel):
    """Data model for tagging."""

    text: str
    language: str
    sentiment: str
    toxicity: float
    hate: float
    hate_threatening: float
    violence: float

Initialize the model

In [3]:
mistral_ollama_llm = llm = Ollama(model="mistral:latest", base_url="http://localhost:11434", temperature=0)

Define ollama pydantic program

In [4]:
mistral_prompt_template = """
<s> [INST] I request you to divide the user comment below into aspects and perform sentiment analysis and give the result in sentiment analysis json format. Just return result in json format. Don't add any other comments.

Note: The toxicity, hate, hate_threatening and violence scores should be in the range (0.1 - 1). The scores should be a float.
The sentiment label should be either ['positive', 'negative', 'neutral'].
The text label should be the comment itself. 
The language label should be a full name of the language in which the comment is written. [/INST] </s> 
[INST] comment: {user_comment} 
Answer: [/INST]
"""
mistral_prompt_tmpl = PromptTemplate(mistral_prompt_template)

mistral_program = LLMTextCompletionProgram(
    output_parser=PydanticOutputParser(Tag),
    prompt=mistral_prompt_tmpl,
    llm=mistral_ollama_llm,
    verbose=True,
)

Run the tagging program to get get structured output

In [5]:
mistral_output = mistral_program(user_comment="I love this world!")

In [6]:
mistral_output.model_dump()

{'text': 'I love this world!',
 'language': 'English',
 'sentiment': 'positive',
 'toxicity': 0.0,
 'hate': 0.0,
 'hate_threatening': 0.0,
 'violence': 0.0}

In [7]:
mistral_output = mistral_program(user_comment="Je n'aime pas ce film, c'est le pire film jamais réalisé !")

In [8]:
mistral_output.model_dump()

{'text': "Je n'aime pas ce film, c'est le pire film jamais réalisé !",
 'language': 'French',
 'sentiment': 'negative',
 'toxicity': 0.3,
 'hate': 0.6,
 'hate_threatening': 0.1,
 'violence': 0.1}